In [30]:
import modules.singleton_reduction as singleton
import modules.isolate_reduction as isolate
from tqdm import tqdm
import importlib
importlib.reload(singleton)
importlib.reload(isolate)
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor
import io
import copy


In [23]:
# Load the full graph
edges = pd.read_csv("../data/edges_updated_reversed.csv", sep=' ')
edges_array = np.array(edges.loc[:, ['Source', 'Target']])
nodes = pd.read_csv(r"../data/nodes.csv", sep=",")
nodes_array = nodes["# index"]

G_full = nx.DiGraph()
G_full.add_edges_from(edges_array)
G_full.add_nodes_from(nodes_array)

In [103]:
importlib.reload(singleton)
importlib.reload(isolate)

G = copy.deepcopy(G_full)
G = singleton.singleton_reduction(G)
G_full_sr = copy.deepcopy(G)
# isolatedNodes = list(nx.isolates(G))
G, isolatedNum = isolate.isolate_reduction(G)

Node count before singleton reduction:  58742
Nodes before:  52124
Nodes after:  19257


In [104]:
import modules.sir_threaded as sir
import importlib
import copy
importlib.reload(sir)
init_infected = [9]
max_steps = 100
G_temp = copy.deepcopy(G)

# Uncomment below block if you want to only view nodes that are accessible from init_infected
G_temp = G_temp.subgraph(sir.get_accessible_sus_nodes(G_temp, init_infected)).copy()

In [ ]:
# Uncomment below block if you want to only view nodes that are accessible from init_infected
# print("Nodes to calculate: ", len(G_temp))
# pos = nx.spring_layout(G_temp, seed=8020, gravity=0.75)

In [105]:
importlib.reload(sir)

_, _, _, _, infotext_total, constants_total = sir.sir_model(G_temp, G_full_sr, init_infected=init_infected, infection_rate=0.2, recovery_rate=0.1, max_steps=max_steps, doSingletonReduction=True, noticeability_rates=(0.0001,0.01), network_type='full')

[DEBUG2] False
Running model...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Step:  1
[DEBUG] QT:  0
[INFO] Infecting singletons and neighbors + Recovering infected nodes...
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG] Added 10 loss due to: Infection creation
[DEBUG

In [60]:
infotext_total

[{'ssw': 950,
  'sso': 5668,
  'it': 1,
  'st': 4990,
  'rt': 0,
  'qt': 0,
  'ss': 6618,
  'is': 0,
  'rs': 0},
 {'ssw': 801,
  'sso': 5668,
  'it': 617,
  'st': 4374,
  'rt': 0,
  'qt': 0,
  'ss': 6469,
  'is': 149,
  'rs': 0},
 {'ssw': 667,
  'sso': 5668,
  'it': 1148,
  'st': 3843,
  'rt': 0,
  'qt': 0,
  'ss': 6335,
  'is': 283,
  'rs': 0},
 {'ssw': 576,
  'sso': 5668,
  'it': 1580,
  'st': 3411,
  'rt': 0,
  'qt': 0,
  'ss': 6244,
  'is': 374,
  'rs': 0},
 {'ssw': 493,
  'sso': 5668,
  'it': 1955,
  'st': 3036,
  'rt': 0,
  'qt': 0,
  'ss': 6161,
  'is': 457,
  'rs': 0},
 {'ssw': 423,
  'sso': 5668,
  'it': 2300,
  'st': 2691,
  'rt': 0,
  'qt': 0,
  'ss': 6091,
  'is': 527,
  'rs': 0},
 {'ssw': 359,
  'sso': 5668,
  'it': 2607,
  'st': 2384,
  'rt': 0,
  'qt': 0,
  'ss': 6027,
  'is': 591,
  'rs': 0},
 {'ssw': 315,
  'sso': 5668,
  'it': 2895,
  'st': 2096,
  'rt': 0,
  'qt': 0,
  'ss': 5983,
  'is': 635,
  'rs': 0},
 {'ssw': 273,
  'sso': 5668,
  'it': 3102,
  'st': 1889,
  'rt

In [58]:
constants_total

{'tn': 58742, 'xt': 53751, 'virusFoundStep': 61}

In [108]:
# Graphical Analysis of a single run
plt.close()

# create data
x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

loss_total = [i['loss'] for i in infotext_total]
max_loss_total = max(loss_total)
loss_total = [i/max_loss_total*(constants_total['tn']-constants_total['xt']) for i in loss_total]


# plot lines
plt.plot(x, st_total, label = "Total susceptible", alpha=0.5, color='g')
plt.plot(x, it_total, label = "Total infected", alpha=0.5, color='r')
plt.plot(x, rt_total, label = "Total recovered", alpha=0.5, color='b')
plt.plot(x, qt_total, label = "Total quarantined", alpha=0.5, color='0.7')
plt.plot(x, loss_total, label = "Loss (scaled)", alpha=0.5, color='k')
plt.vlines(constants_total['virusFoundStep'], 0, constants_total['tn']-constants_total['xt'], color='k', linestyle='dotted', lw=1)
plt.legend()
plt.savefig(f"results/test.png", format="PNG", bbox_inches='tight')

In [95]:
# Loss analysis of a single run
plt.close()
loss = {'quarantine': 1, 'infected': 1}

x = range(len(infotext_total))
it_total = [i['it'] for i in infotext_total]
st_total = [i['st'] for i in infotext_total]
rt_total = [i['rt'] for i in infotext_total]
qt_total = [i['qt'] for i in infotext_total]

plt.stackplot(x, (
                    [i*loss['infected'] for i in it_total],
                    [i*loss['quarantine'] for i in qt_total]
                ),
             labels=['Loss from infected', 'Loss from quarantines'],
             colors=['#ff0000','#444444'],
              alpha=0.8)
plt.legend()
plt.savefig(f"results/test2.png", format="PNG", bbox_inches='tight')

In [130]:
importlib.reload(sir)

# Loss analysis of multiple runs
run_count = 100

init_infected = [9]
# init_infected = 
max_steps = 1000
infection_rate = 0.2
recovery_rate = 0.1
noticeability_rates = (0.0001,0.01)
quarantine_length = 0
contact_tracing = 1
loss_matrix = ((1, 0),(0.2, 0.1))


loss_results = []
for run in range(run_count):
    G_temp = copy.deepcopy(G)
    # Uncomment below block if you want to only view nodes that are accessible from init_infected
    G_temp = G_temp.subgraph(sir.get_accessible_sus_nodes(G_temp, init_infected)).copy()

    temp_packed = sir.sir_model(G_temp, G_full_sr, 
                                init_infected=init_infected, 
                                infection_rate=infection_rate, 
                                recovery_rate=recovery_rate, 
                                max_steps=max_steps, 
                                doSingletonReduction=True, 
                                noticeability_rates=noticeability_rates, 
                                quarantine_length=quarantine_length,
                                contact_tracing=contact_tracing,
                                loss_matrix=loss_matrix,
                                network_type='full',
                                disablePrintOutput=True
                                )
    _, _, _, _, infotext_total, constants_total = temp_packed

    print("Run: ", run, " --- Steps taken: ", len(infotext_total), " --- Loss: ", infotext_total[-1]['loss'])
    loss_results.append(infotext_total[-1]['loss'])

(np.mean(loss_results), np.std(loss_results))

Run:  0  --- Steps taken:  98  --- Loss:  18614.000000000015
Run:  1  --- Steps taken:  88  --- Loss:  19956.000000000007
Run:  2  --- Steps taken:  91  --- Loss:  11978.00000000002
Run:  3  --- Steps taken:  132  --- Loss:  15093.800000000017
Run:  4  --- Steps taken:  72  --- Loss:  8909.0
Run:  5  --- Steps taken:  79  --- Loss:  13455.199999999995
Run:  6  --- Steps taken:  126  --- Loss:  17060.00000000003
Run:  7  --- Steps taken:  87  --- Loss:  11881.800000000001
Run:  8  --- Steps taken:  98  --- Loss:  13249.400000000005
Run:  9  --- Steps taken:  100  --- Loss:  12026.400000000001
Run:  10  --- Steps taken:  97  --- Loss:  16707.199999999986
Run:  11  --- Steps taken:  103  --- Loss:  12588.00000000002
Run:  12  --- Steps taken:  82  --- Loss:  14030.800000000003
Run:  13  --- Steps taken:  102  --- Loss:  16727.999999999996
Run:  14  --- Steps taken:  102  --- Loss:  19589.999999999985
Run:  15  --- Steps taken:  93  --- Loss:  10583.999999999998
Run:  16  --- Steps taken: 

(14869.428000000004, 3535.8241905411546)

# loss_matrix = ((1, 0),(0.2, 0.1))
## max_steps = 100
quarantine_length 5: 15342.928
quarantine_length 5, ct level 2: 14599.878000000002
quarantine_length 10: 15261.373
quarantine_length 5, ct level 1, nt_rate * 10 noticed: 14576.289000000004
quarantine_length inf, ct level 1: 20178.598

## max_steps = 1000 (*)
quarantine_length 5, ct level 1: 14514.140000000009
quarantine_length 10, ct level 1: 15413.901000000007
quarantine_length 5, ct level 2: 14708.250000000007
quarantine_length 5, ct level 3: 15418.860000000004
quarantine_length 2, ct level 1: 15211.508000000003
no quarantines: 14869.428000000004